In [ ]:
%%capture
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
from kaggle_environments.envs.kore_fleets.kore_fleets import balanced_agent
env = make("kore_fleets", debug=True)
steps = env.run([balanced_agent, balanced_agent])
env.render(mode="ipython", width=800, height=640)

In [ ]:
import sys
sys.path.append("../input/kore-pixyz-library")
from steps_analyze import steps_analyze
from steps_compare import steps_compare

steps_analyze(steps)

pixyz  
last update 2022 06 04  
ゆっくりしていってね！  

<img src="https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 300>

## Contents

<span style="font-size: 200%; color: black;">
    
* **v0** [**Official guide**](#v0-Official_guide)

* **v1** [**Github & Library**](#v1-Github-&-Library)

* **v2** [**step analysis**](#v2-step_analysis)

</span>

**霊夢:まずは、公式ガイドの日本語訳から始めるぜ**

https://www.kaggle.com/code/bovard/kore-intro-i-the-basics

# v0 Official_guide

**霊夢:今回はコンペ理解のために、主催者が書いてくれたコードを日本語訳してみたよ。**

**魔理沙:他に気になったことがあったら突っ込んでいくぞ。**

**Reimu: This time, I translated the code written by the organizer into Japanese to understand the competition.**

**Marisa: If you have any other concerns, I'll dig in.**

## Welcome Commander!

このノートブックでは、Kore艦隊のルールと環境を紹介します。

This notebook will give you an introduction to the Kore Fleet rules and environment

## Let's start with the shipyard

造船所は、アクションを割り当てる唯一のユニットです。 毎ターン、一つの造船所でどちらかをすることができます 

1. 船を建造する。
2. 艦隊を進水させる。

では実際に、造船所がどのようになものかを見てみましょう

The shipyard it the only unit you assign actions to. You can either

1. build more ships
2. launch a fleet of ships

but for now let's just see what a shipyard looks like

In [ ]:
%%writefile do_nothing.py
# First we will make a do_nothing player to observe the game board
def do_nothing():
    pass

In [ ]:
env.run(["/kaggle/working/do_nothing.py"])
env.render(mode="ipython", width=800, height=640)

# It did nothing!
あなたの造船所（青色）は、0隻の船を持っています。（左上隅の0で示されています）、
このままでは、何もしません。


1. KoreのあるマスのKore量が増えました（星が大きくなりました）

2. 造船所の右下の数はゆっくりと10に増えました。この数は、その造船所でそのターンを建造できる船の最大数を表しています。 造船所が交代するたびに、この数字は0にリセットされます！

Your shipyard (in blue) had 0 ships the entire game (denoted by the 0 in the upper left hand corner) and did nothing.

Notice a few things did happen!

1. the kore amount on the tiles with kore grew (the stars got bigger)
2. the number in the bottom righthand tile grew slowly to 10
This number represents the number of ships that can be built a turn at that shipyard! This number is reset to 0 every time a shipyard changes hands!

**魔理沙:　画面左側に表示されているKoreは造船所が持っている資源の数、Cargoは艦隊が持っている資源の数、Shipsは味方全体が持っている船のかずだぜ。**

**霊夢:　フィールドに散らばっている資源はターンが経過すると大きくなるんだね。どんな割合で大きくなっていくんだ？**

**魔理沙:　Koreはそのマスに艦隊がいなかった場合、毎ターン2％増加してくぜ。**

**Marisa: Kore displayed on the left side of the screen is the number of resources owned by the shipyard, Cargo is the number of resources owned by the fleet, and Ships is the number of ships owned by all allies.**

**Reimu: The resources scattered in the field grow larger as the turn passes. At what rate will it grow?**

**Marisa: Kore will increase by 2% each turn if there is no fleet in that square.**

## Building some ships

それでは、毎ターン船を建造しようとする簡単なエージェントを作りましょう！

船の建造には10Koreの費用がかかり、Koreはゲーム開始時500あるので、50ターン船の建造が行われるでしょう。

Now let's make a simple player that tries to build a ship every turn!

Note that since ships cost 10 kore to build and we start with 500, we will expect to build ships for 50 turns!

In [ ]:
%%writefile builder.py
# this one builds ships!

from kaggle_environments.envs.kore_fleets.helpers import *

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    
    # loop through all shipyards you control
    for shipyard in me.shipyards:
        # build a ship!
        if kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/builder.py"])
env.render(mode="ipython", width=800, height=640)

**霊夢：一つの造船所で1ターンに船は何隻造船できるんだ？**

**魔理沙:造船所ができてからターンが経過するごとに作れる船の数が増えていって、最大で1ターンに10隻まで作ることができるぜ。具体的には、造船所が現在Y隻の船をスポーンできるようになったとすると、それからY^2 + 1ターン経過した後、Y+1隻の船を生産できるようになるぜ。次の表をみてくれだぜ。**

**Reimu: How many ships can be built in one turn at one shipyard?**

**Marisa: The number of ships that can be built increases with each turn since the shipyard was built, and you can build up to 10 ships per turn. Specifically, if the shipyard is now able to spawn Y ships, then Y ^ 2 + 1 turn later will be able to produce Y + 1 ships. See the table below.**

| Turns Controlled | Spawn Maximum | 
| ---- | ---- |
| 0 | 1 |
| 2 | 2 |
| 7 | 3 |
| 17 | 4 |
| 34 | 5 |
| 60 | 6 |
| 97 | 7 |
| 147 | 8 |
| 212 | 9 |
| 294 | 10 |

# We built 50 ships!

造船所の左上にある数字が50に増え、Koreがなくなると止まります。 UIは、青いプレーヤーにも50隻の船を制御しないことを示していることに注意してください。

You saw the number in the upper left of the shipyard increment to 50 and stop when we ran out of kore! Note that the ui also shows the blue player also no contorls 50 ships.

# To the stars!

次に、艦隊を進水させてみましょう。 艦隊は一連の指示で発射されます（北南東西のいづれかに、毎ターン移動します）。 とりあえず北に艦隊を発射しましょう。

この艦隊が途中でKoreを拾い、より多くの船を建造できるようになるでしょう！

Next lets try launching a fleet. A fleet is launched with a series of instructions (go North, South, East, West and follows those, moving every turn). For now let's now worry about that and just launch a fleet to the north.

We expect this fleet to pick up some kore along the way, enabling us to build more ships!

In [ ]:
%%writefile launcher.py
   
from kaggle_environments.envs.kore_fleets.helpers import *

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
        elif shipyard.ship_count > 0:
            direction = Direction.NORTH
            action = ShipyardAction.launch_fleet_with_flight_plan(2, direction.to_char())
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/launcher.py"])
env.render(mode="ipython", width=800, height=640)

**魔理沙:艦隊は、造船所から発射されたターンにルートが決まっていて、後から変更することができないって事か？**

**Marisa: Does the fleet have a route on the turn it fires from the shipyard and can't be changed later?**

# Flight Control

今、あなたはこの戦略の弱点に気付いているかもしれません、これではKoreが効率的に回収できません！ 私たちは、これまで艦隊が行ったことのない場所に艦隊が行くことを可能にする飛行計画を立てなければなりません！

Now you might notice a certain weakness of our strategy, it doesn't mine out any of the board! Now we have to make a flight plan which allows our fleet go where no fleet has ever gone before!

In [ ]:
from random import randint
randint(2, 9)

In [ ]:
%%writefile pilot.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count >= 50:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(50, flight_plan)
            shipyard.next_action = action
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/pilot.py"])
env.render(mode="ipython", width=800, height=640)

52ターン目にUIを一時停止すると、艦隊が進水するフライトプランを確認できます。

Notice that if you pause the UI on turn 50, you can see the flight plan your fleet will launch with.

## How flight plans work

1. NESW方向の場合は、進水の方向を変更して一致させます
2. 数値の場合は、数値をデクリメントします

1. if it's a direction NESW, change the fleet direction to match
2. if it's a number, decrement the number

examples:

#### N2S (go north, then continue for 2 squares, then go south)
```
N2S
2S
1S
S
(fleet will continue south)
```

#### N10E (go north, then continue for 10 squares, then go east)
```
N10E
10E
9E
8E
7E
6E
5E
4E
3E
2E
1E
E
(fleet will continue east)
```

# v1 Github & Library

霊夢:githubにkaggle_environmentsがあるから、その中の/envs/kore_fleetsを見よう

https://github.com/Kaggle/kaggle-environments/tree/master/kaggle_environments/envs/kore_fleets

**霊夢:色々なagentをpixyz libraryに入れているから、それを使っていくよ**

**魔理沙:sys.path.appendして、importすれば使えるぜ。**

In [ ]:
from do_nothing import do_nothing
env = make("kore_fleets",configuration={"randomSeed":42}, debug=True)
env.run([do_nothing])
env.render(mode="ipython", width=800, height=640)

**霊夢:makeの時にrandomSeedを指定してあげると、毎回同じマップを表示することができるね。**

**魔理沙:こういったことも、githubを読むとよくわかるから、大切だぞ**

**Reimu: If you specify randomSeed at the time of make, you can display the same map every time.**

**Marisa: It's important because you can understand these things by reading github**

**魔理沙:kaggle_environmentsの中にはagentもいくつか入っているから、呼び出して戦わせることもできるぜ。**

In [ ]:
from kaggle_environments.envs.kore_fleets.kore_fleets import miner_agent,attacker_agent
steps = env.run([miner_agent,attacker_agent])
env.render(mode="ipython", width=800, height=640)

**霊夢:Helper functionの中身が知りたい**

**魔理沙:inspectを使えば良いんじゃね？**

**Reimu: I want to know the contents of the Helper function**

**Marisa: Should I use inspect?**

**霊夢:Apache License, Version 2.0だったのでありがたく公開したよ。**

**魔理沙:中身を見ていこうか。**

Observation
* kore (星の位置)
* players (敵味方含めたプレイヤーの情報)
* player (自分のプレイヤーID)

Configuration
* agent_timeout (タイムアウトになる時間)  
* starting_kore (開始時に所持しているKoreの量)  
* size (フィールドのサイズ)  
* spawn_cost (造船に必要なKoreの量)  
* convert_cost (造船所を作る為に必要な船の数)  
* regen_rate (艦隊が発掘できる資源の割合)  
* max_cell_kore (星が持つ資源の最大量)  
* random_seed (seed値)  

Board
* configuration (ゲーム設定)
* players (敵味方含めたプレイヤーの情報)
* fleets (艦隊の情報)
* shipyards (造船所の情報)
* cells (それぞれのマスの情報)
* step (現在のターン数）
* current_player_id (自身のプレイヤーID)

**霊夢:最後に、beta版のチャンピオンのagentをbalanced_agentと戦わせてみよう。**

In [ ]:
sys.path.append("../input/kore-beta-1st-place-solution")
from main import agent as beta_champion
steps = env.run([beta_champion,balanced_agent])
env.render(mode="ipython", width=800, height=640)

# v2 step_analysis

**霊夢:env.runを変数にstepsに代入して、中身を見てみるよ。**

**魔理沙:さっきのstepsをそのまま使うぜ。**

**Reimu: Substitute env.run into a variable for steps and take a look at the contents.**

stepsを出力先にすると、データの中身を見ることができます。

In [ ]:
print(type(steps))
print(len(steps))

stepsはlist形式で、勝負が167ターンで決まったので、stepsの数は167です。

まず、100ターン目のデータを見てみます。

In [ ]:
print(steps[100])

1つのstepはagentの数だけあるlist形式で、辞書型になっている。重複するobservationは2つ目以降は省略されている。

## step list
* action 辞書型
* reward　float
* info 辞書型
* observation 辞書型
 * kore リスト
 * players リスト(shipとshipyard,船の数,koreの数,x座標,y座標,航路)
 * player int
 * step int
 * remainingOverageTime int
 * status ACTIVE

**次は、observationについて解析してみます**

**Next, let's analyze observation**

In [ ]:
for key, item in steps[100][0]['observation'].items():
    print(key,item)

## kore

In [ ]:
import numpy as np

In [ ]:
kore = steps[0][0]['observation']['kore']
len(kore)

In [ ]:
kore = np.array(kore).reshape((21,21))
print(kore)

**.reshapeを使って、koreを21*21のリストに変換します。**

**Use .reshape to convert kore to a list of 21 * 21.**

In [ ]:
print(kore[9:12,9:12])
print(kore[9:12,9:12].sum())

**範囲を指定してその範囲のkoreを取得し、.sum()でその範囲の値の合計を求めることができます。**

**You can specify a range to get the kore of that range and use .sum () to sum the values in that range.**

In [ ]:
print(np.roll(kore,(3,4),axis = (0,1)))

**.rollを使用して、koreのリストを指定した数だけ回転させます。**

**Use .roll to rotate the list of kore by the specified number.**

In [ ]:
print(kore)

**元のデータは変更されません**

**Original data will not change**

## players

**playersの中には、それぞれのプレイヤーの情報が入ったリストが入っています。それぞれのリストには、プレイヤーのKore(float)、shipyardの情報（辞書型）、shipの情報（辞書型）、が入っています。shipyardの情報は、ID、position、船の数、経過ターンが入ってます。**

In [ ]:
steps[100][0]['observation']['players']

0.026999999999986812 reward

'0-1': [110, 14, 100]  
'0-1' shipyard ID  
110   position(21×y + x)  
14    ship count  
100   elapsed turn  

'83-1': [172, 108.26799999999997, 21, 0, '3E']  
'83-1' fleet ID
172    position(21×y + x)  
108.   Cargo  
21     ship count  
0      direction  
'3E'   flight plan